In [40]:
import pandas as pd
import mygene

In [51]:
def load_and_filter_data(file_path):
    # Load data from the file
    df = pd.read_excel(file_path)
    # Create a copy of the DataFrame
    temp_df = df.copy()

    # print how many rows in df_nm have a p-value lower than 0.05
    print(temp_df[temp_df['P-value (500nM vs T)'] < 0.05].shape)

    # In column 'Gbkey' only keep rows with value: 'Gene | mRNA'
    temp_df = temp_df[temp_df['Gbkey'] == 'Gene | mRNA']
    # Remove column 'Gbkey'
    temp_df.drop(columns=['Gbkey'], inplace=True)
    
    # remove any row if the value in column gene_symbol does not start with a letter
    temp_df = temp_df[temp_df['Gene_Symbol'].str[0].str.isalpha()]
    # turn all values in Human_Name column to uppercase if they are letters, some values are a combination of letters and numbers
    temp_df['Gene_Symbol'] = temp_df['Gene_Symbol'].str.upper()
    
    # check how many duplicate values in Gene_Symbol column
    print('Number of duplicate values in Gene_Symbol column: ', temp_df.duplicated(subset=['Gene_Symbol']).sum())
    
    # create list of all Human_Name values
    query_list = temp_df['Gene_Symbol'].tolist()
    mg = mygene.MyGeneInfo()
    
    df = mg.querymany(query_list, scopes='symbol', fields='symbol,entrezgene', species='human', entrezonly=True, verbose=True, as_dataframe=True)
    # Rename the columns in df for clarity
    df.rename(columns={'symbol': 'Gene_Symbol', 'entrezgene': 'GeneID'}, inplace=True)
    
    # Merge temp_df with df to get the GeneID
    merged_df = pd.merge(temp_df, df[['Gene_Symbol', 'GeneID']], on='Gene_Symbol', how='left')
    
    # create df from the dict_list
    print(merged_df.shape)
    print(merged_df.head())
    # check if df has duplicates
    print('Number of duplicate values in Gene_Symbol column: ', merged_df.duplicated(subset=['GeneID']).sum())
    # change order of columns
    merged_df = merged_df[['GeneID', 'Gene_Symbol', 'P-value (T vs N)', 'Score (T vs N)', 'P-value (500nM vs T)', 'Score (500nM vs T)']]
    # change column name from Gene_Symbol to Human_Name
    merged_df.rename(columns={'Gene_Symbol': 'Human_Name'}, inplace=True)
    return merged_df


In [90]:
def load_idmap(file_path):
    id_file = pd.read_excel(file_path)
    id_file_copy = id_file.copy()
    # # print file name 
    # print(file_path)
    # print(id_file.shape)
    # # print how many duplicate rows there are in the DataFrame
    # print('duplicates in the DataFrame Before drop:', id_file.duplicated().sum())
    # # delete rows that are duplicates
    # id_file.drop_duplicates(inplace=True)
    # print('duplicates in the DataFrame After drop:', id_file.duplicated().sum())
    # print(id_file.shape)
    # # print how many duplicates in the query column
    # print('duplicates in the Human_Name column:', id_file['Human_Name'].duplicated().sum())
    # print('duplicates in the GeneID column:', id_file['GeneID'].duplicated().sum())
    # # print all unique duplicates in the GeneID column
    # print('unique duplicates in the GeneID column:', id_file['GeneID'][id_file['GeneID'].duplicated()].unique())
    # # # drop duplicates from the query column
    # # id_file.drop_duplicates(subset=['query'], inplace=True)
    # # # print how duplicated in the query column after dropping duplicates
    # # print('duplicates in the query column after dropping duplicates:', id_file['query'].duplicated().sum())
    #use mygene to get the gene ID for the gene symbol
    mg = mygene.MyGeneInfo()
    # create list of all Human_Name values
    query_list = id_file['Human_Name'].tolist()
    dict_list = mg.querymany(query_list, scopes='symbol', species='human', as_dataframe=True)
    # create df from the dict_list
    df = pd.DataFrame(dict_list)
    print(df.shape)
    print(df.head())
    return df

In [74]:
def merge_dataframes(df1, df2):
    # Merge the two DataFrames
    merged_df = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)

    # print how many rows and columns are in the DataFrame
    print(merged_df.shape)
    # print how many rows have NaN values in the GeneID column
    print(merged_df['GeneID'].isna().sum())
    # print how many rows have NaN values in the Human_Name column
    print(merged_df['Human_Name'].isna().sum())
    print('duplicates in the Human_Name column:', merged_df['Human_Name'].duplicated().sum())
    print('duplicates in the GeneID column:', merged_df['GeneID'].duplicated().sum())
    # print how many whole rows are duplicated
    print('duplicates in the DataFrame:', merged_df.duplicated().sum())
    return merged_df

In [52]:
# Usage
dir_path = 'Inputs/experiments_data/Parkinson/'
data_file_path = dir_path + 'Parkinson_t_v_n_500nm_v_t.xlsx'
temp_data_file = load_and_filter_data(data_file_path)
# save the filtered data
temp_data_file.to_excel(dir_path + 'Parkinson_t_v_n_500nm_v_t_filtered.xlsx', index=False)
# print how many rows and columns are in the DataFrame
print(temp_data_file.shape)
print(temp_data_file.head())

(415, 6)
Number of duplicate values in Gene_Symbol column:  0


KeyboardInterrupt: 

In [43]:
# print how many nan values are in the GeneID column
print(temp_data_file['GeneID'].isna().sum())
# remove rows with nan values in the GeneID column
temp_data_file.dropna(subset=['GeneID'], inplace=True)
# print how many duplicate values are in the GeneID column
print('duplicates in the GeneID column:', temp_data_file['GeneID'].duplicated().sum())
# print the unique duplicate values in the GeneID column
print('unique duplicates in the GeneID column:', temp_data_file['GeneID'][temp_data_file['GeneID'].duplicated()].unique())
# save the filtered data
temp_data_file.to_excel(dir_path + 'Parkinson_t_v_n_500nm_v_t_filtered.xlsx', index=False)


1421
duplicates in the GeneID column: 0
unique duplicates in the GeneID column: []


In [46]:
dir_path = 'Inputs/experiments_data/Parkinson/'
T_v_N_file = dir_path + 'roded_T_v_N.xlsx'
nm_file = dir_path + 'roded_500nm.xlsx'

# read the files
df_T_v_N = pd.read_excel(T_v_N_file)
df_nm = pd.read_excel(nm_file)

# print how many rows in df_nm have a p-value lower than 0.05
print(df_nm[df_nm['P-value'] < 0.05].shape)

# print how many rows and columns are in the DataFrames
print(df_T_v_N.shape)
print(df_nm.shape)

# print how many duplicate values are in the GeneID column
print('duplicates in the GeneID column:', df_T_v_N['GeneID'].duplicated().sum())
print('duplicates in the GeneID column:', df_nm['GeneID'].duplicated().sum())

# print the unique duplicate values in the GeneID column
print('unique duplicates in the GeneID column:', df_T_v_N['GeneID'][df_T_v_N['GeneID'].duplicated()].unique())
print('unique duplicates in the GeneID column:', df_nm['GeneID'][df_nm['GeneID'].duplicated()].unique())

# how many nan values are in the Human_Name column
print(df_T_v_N['Human_Name'].isna().sum())

# Convert 'GeneID' in df_T_v_N to string format
df_T_v_N['GeneID'] = df_T_v_N['GeneID'].astype(str)

# Iterate over df_T_v_N to update 'Human_Name'
for index, row in df_T_v_N.iterrows():
    if pd.isna(row['Human_Name']) and row['GeneID'] in temp_data_file['GeneID'].values:
        # Find the corresponding 'Human_Name' from temp_data_file using 'GeneID'
        matching_row = temp_data_file[temp_data_file['GeneID'] == row['GeneID']]
        if not matching_row.empty:
            # Update the 'Human_Name' in df_T_v_N
            df_T_v_N.at[index, 'Human_Name'] = matching_row['Human_Name'].iloc[0]
# print how many duplicate values are in the GeneID column
print('duplicates in the GeneID column:', df_T_v_N['GeneID'].duplicated().sum())

# print the unique duplicate values in the GeneID column
print('unique duplicates in the GeneID column:', df_T_v_N['GeneID'][df_T_v_N['GeneID'].duplicated()].unique())

# how many nan values are in the Human_Name column
print(df_T_v_N['Human_Name'].isna().sum())

# save t_v_n data
df_T_v_N.to_excel(dir_path + 'roded_T_v_N_filtered.xlsx', index=False)


(109, 4)
(14274, 4)
(14274, 4)
duplicates in the GeneID column: 0
duplicates in the GeneID column: 0
unique duplicates in the GeneID column: []
unique duplicates in the GeneID column: []
93
duplicates in the GeneID column: 0
unique duplicates in the GeneID column: []
93


In [91]:
file_path_idmap = 'Data/H_sapiens/gene_names/ID_to_Name_Map.xlsx'
df_idmap = load_idmap(file_path_idmap)
file_path_idmap = 'Data/H_sapiens/gene_names/Copy_of_idmap.xlsx'
df_idmap1 = load_idmap(file_path_idmap)
df_merged = merge_dataframes(df_idmap, df_idmap1)
df_merged.to_excel('Data/H_sapiens/gene_names/idmap_merged.xlsx', index=False)

9 input query terms found dup hits:	[('ARMCX5-GPRASP2', 2), ('BORCS8-MEF2B', 2), ('BUB1B-PAK6', 2), ('CRYZL2P-SEC16B', 2), ('MICOS10-NBL
118 input query terms found no hit:	['BTBD11', 'C19ORF54', 'C20ORF27', 'SPATA5', 'SPATA5L1', 'C1ORF109', 'FAM172A', 'RNF165', 'C18ORF25'


(15662, 7)
           _id     _score entrezgene  \
query                                  
A1BG         1  19.289219          1   
A1CF     29974  17.584267      29974   
A2M          2  19.319730          2   
A2ML1   144568  18.956692     144568   
A4GALT   53947  18.395320      53947   

                                                     name  symbol   taxid  \
query                                                                       
A1BG                               alpha-1-B glycoprotein    A1BG  9606.0   
A1CF                       APOBEC1 complementation factor    A1CF  9606.0   
A2M                                 alpha-2-macroglobulin     A2M  9606.0   
A2ML1                        alpha-2-macroglobulin like 1   A2ML1  9606.0   
A4GALT  alpha 1,4-galactosyltransferase (P1PK blood gr...  A4GALT  9606.0   

       notfound  
query            
A1BG        NaN  
A1CF        NaN  
A2M         NaN  
A2ML1       NaN  
A4GALT      NaN  


496 input query terms found dup hits:	[('ABCD1', 2), ('ACD', 2), ('ADAM12', 2), ('ADAM1A', 2), ('ADCY8', 2), ('ADRA1D', 2), ('ADSL', 2), (


(13983, 6)
            _id     _score entrezgene  \
query                                   
A2M           2  19.329280          2   
A2ML1    144568  18.965952     144568   
A3GALT2  127550  18.735895     127550   
A4GALT    53947  18.396685      53947   
A4GNT     51146  18.224072      51146   

                                                      name   symbol  taxid  
query                                                                       
A2M                                  alpha-2-macroglobulin      A2M   9606  
A2ML1                         alpha-2-macroglobulin like 1    A2ML1   9606  
A3GALT2                  alpha 1,3-galactosyltransferase 2  A3GALT2   9606  
A4GALT   alpha 1,4-galactosyltransferase (P1PK blood gr...   A4GALT   9606  
A4GNT            alpha-1,4-N-acetylglucosaminyltransferase    A4GNT   9606  
(27792, 7)


KeyError: 'GeneID'

In [67]:
df_joined = join_dataframes(temp_data_file, df_idmap)
df_final = handle_duplicates(df_joined)

# Save the final DataFrame
df_final.to_excel('final_output_iPSC.xlsx', index=False)

NameError: name 'join_dataframes' is not defined